# Tutorial 9: Multi-Agent Workflows

**Difficulty:** Intermediate | **Time:** 60 minutes

## Learning Objectives

- Design and implement multi-agent workflows
- Coordinate multiple agents with GraphState
- Route work between specialized agents
- Aggregate results from parallel agent execution
- Handle agent failures and fallbacks

## Real-World Use Case

Single agents are powerful, but complex tasks often require a team of specialized agents working together. Think of a software development team: you have a Product Manager defining requirements, Developers writing code, and QA Engineers testing it. No single person does it all perfectly.

In this tutorial, you'll learn how to orchestrate a "team" of AI agents to handle complex workflows like:
- **Research Pipeline**: Researcher -> Analyst -> Writer
- **Customer Support**: Triage -> Specialist (Technical/Billing) -> QA
- **Panel of Experts**: Multiple experts giving diverse opinions which are then summarized



## Core Concepts

### 1. Agent Specialization
Instead of one giant prompt doing everything, we create specialized agents with focused system prompts. This improves performance, maintainability, and debuggability.

### 2. Workflow Patterns
- **Sequential**: Agents work in a chain (Output of A -> Input of B)
- **Parallel**: Agents work simultaneously (Fan-out), results are combined (Fan-in)
- **Router**: A "boss" agent decides which specialist should handle the task

### 3. GraphState Coordination
While nodes can pass data directly, **GraphState** acts as a shared whiteboard where agents can read the full context and write their contributions.



## Setup

Let's import the necessary classes. We'll need `GraphState` for sharing data between agents.



In [1]:
import asyncio
from spark.agents import Agent, AgentConfig
from spark.models import OpenAIModel
from spark.graphs import Graph, GraphState
from spark.nodes import Node, EdgeCondition
from spark.utils.common import arun

# make sure to `export OPENAI_API_KEY=sk-...` before running notebook
default_model = OpenAIModel(model_id='gpt-5-mini')

## Example 1: Sequential Workflow (The Content Pipeline)

Let's build a classic sequential pipeline: **Idea -> Researcher -> Writer**.

We'll use `GraphState` to store the evolving content.



In [2]:
# 1. Define the Agents

class Researcher(Agent):
    async def process(self, context):
        outputs = await self.work(context)
        return {'research_notes': outputs.content}


class Writer(Agent):
    async def process(self, context):
        outputs = await self.work(context)
        await context.graph_state.set('blog_post', outputs.content)


researcher = Researcher(
    config=AgentConfig(
        model=default_model,
        system_prompt="You are a research assistant. Given a topic, provide 3 key bullet points of facts.",
        prompt_template="Research topic: {{topic}}"
    )
)

writer = Writer(
    config=AgentConfig(
        model=default_model,
        system_prompt="You are a blog writer. Turn the provided research notes into a short, engaging paragraph.",
        prompt_template="Here are the research notes:\n{{research_notes}}"
    )
)

researcher >> writer

pipeline = Graph(start=researcher)

print("=== Running Content Pipeline ===")
await pipeline.run({"topic": "The benefits of walking daily"})

# Retrieve result from state
final_post = await pipeline.get_state("blog_post")
print(f"\n📰 Final Blog Post:\n{final_post}")

=== Running Content Pipeline ===
context dir <class 'spark.graphs.graph_state.GraphState'>

📰 Final Blog Post:
A simple daily walk can do more for your health than you might expect: aiming for about 150 minutes a week (roughly 30 minutes of brisk walking most days) lowers blood pressure, improves cholesterol and cardiorespiratory fitness, and in large studies is linked to a roughly 20–30% reduction in premature death — effects driven by better endothelial function, less inflammation and improved insulin sensitivity. Walking also boosts calorie burn and muscle glucose uptake, helps control weight, lowers diabetes risk and even short post-meal strolls reduce blood sugar and improve HbA1c in people with or at risk for diabetes. On top of that, regular walking eases depression and anxiety, improves sleep and cognition, and preserves muscle strength, balance and joint mobility — reducing fall risk and supporting bone health as we age. Because it’s low-impact, inexpensive and easy to slot in

## Example 2: Router Workflow (Customer Support)

Here we use a **Triage Agent** to decide whether a request is "Technical" or "Billing", and route it to the appropriate specialist.



In [12]:
# 1. Define Agents

tech_support = Agent(
    config=AgentConfig(
        model=default_model,
        system_prompt="You are a friendly technical support engineer. Suggest a troubleshooting step. Limit your answer to under 150 words",
        prompt_template="User Issue: {{issue}}"
    )
)

billing_support = Agent(
    config=AgentConfig(
        model=default_model,
        system_prompt="You are a billing specialist. Explain how to resolve the payment issue. Limit your response to under 250 words",
        prompt_template="User Issue: {{issue}}"
    )
)


class TriageAgent(Agent):
    async def process(self, context):
        outputs = await self.work(context)
        return {'issue': context.inputs.content['issue'], 'category': outputs.content}


triage_agent = TriageAgent(
    config=AgentConfig(
        model=default_model,
        # Force the agent to output just the category name
        system_prompt="Classify the user issue into exactly one of these categories: 'TECHNICAL' or 'BILLING'. Output ONLY the word.",
        prompt_template="Issue: {{issue}}"
    )
)

triage_agent.on(category='TECHNICAL') >> tech_support
triage_agent.on(category='BILLING') >> billing_support

support_graph = Graph(start=triage_agent)

print("=== Test 1: Technical Issue ===")
response = await support_graph.run({"issue": "My screen is flickering when I open the app"})
print(f"Response: {response.content}\n")

print("=== Test 2: Billing Issue ===")
response = await support_graph.run({"issue": "I was charged twice for my subscription"})
print(f"Response: {response.content}")


=== Test 1: Technical Issue ===
Response: Try disabling hardware acceleration: open the app’s Settings → Advanced (or System) and turn off “Use hardware acceleration” (in browsers: Settings → System → “Use hardware acceleration when available”). Fully close the app, restart it (or reboot the device), and see if the flicker stops. If it does, update your graphics driver and the app. If it doesn’t, tell me the app name, device model, and OS so I can suggest the next step.

=== Test 2: Billing Issue ===
Response: Sorry about that — here’s how we’ll fix the duplicate charge.

1. Verify the charge
- Check your bank/credit card statement for merchant name, date, amount, and transaction ID (or last 4 digits of the card).

2. Tell us the details
- Reply with your account email, subscription ID (if known), the two transaction dates/amounts or transaction IDs, and last 4 digits of the card. If the purchase was through Apple/Google, say which store.

3. What I’ll do
- If it’s a duplicate posted c

## Example 3: Parallel Workflow (Panel of Experts)

Sometimes you want multiple opinions on the same topic simultaneously. This is a **Parallel (Fan-out/Fan-in)** pattern.

We will have three experts (Optimist, Pessimist, Realist) analyze a scenario, and then an Aggregator to summarize.



In [ ]:
# 1. Define Experts
expert_prompts = {
    "Optimist": "You are an eternal optimist. Find the best possible outcome.",
    "Pessimist": "You are a careful pessimist. Identify all potential risks and failures.",
    "Realist": "You are a pragmatist. Look at the facts and give a balanced view."
}

experts = [
    Agent(
        name=name, 
        config=AgentConfig(
            model=default_model, 
            system_prompt=prompt,
            prompt_template="Scenario: {{scenario}}"
        )
    )
    for name, prompt in expert_prompts.items()
]

# 2. Define Expert Node
class ExpertNode(Node):
    def __init__(self, agent):
        super().__init__()
        self.agent = agent

    async def process(self, context):
        scenario = await context.graph_state.get("scenario")
        result = await self.agent.run({"scenario": scenario})
        
        # Append opinion to a shared list in GraphState
        # Note: GraphState is thread-safe
        opinion = f"{self.agent.name}: {result.content}"
        print(f"🗣️ {self.agent.name} has spoken.")
        
        async with context.graph_state.transaction() as state:
            if "opinions" not in state:
                state["opinions"] = []
            state["opinions"].append(opinion)
            
        return {"done": True}

# 3. Define Aggregator Node
class AggregatorNode(Node):
    async def process(self, context):
        opinions = await context.graph_state.get("opinions", [])
        print(f"\n📊 Aggregating {len(opinions)} opinions...")
        
        summary_agent = Agent(
            config=AgentConfig(
                model=default_model,
                system_prompt="Summarize the diverse opinions into a cohesive conclusion.",
                prompt_template="Opinions:\n" + "\n".join(opinions)
            )
        )
        
        result = await summary_agent.run({})
        await context.graph_state.set("summary", result.content)
        return {"done": True}

# 4. Build Graph
# Create nodes
expert_nodes = [ExpertNode(agent) for agent in experts]
aggregator = AggregatorNode()

# Create a start node to branch from
start_node = Node()

# Fan-out: Start -> All Experts
for node in expert_nodes:
    start_node >> node

# Fan-in: All Experts -> Aggregator
for node in expert_nodes:
    node >> aggregator

panel_graph = Graph(start=start_node)



In [ ]:
print("=== Panel of Experts ===")
scenario = "We are launching a new AI product next week with minimal testing."
print(f"Scenario: {scenario}\n")

await panel_graph.run(initial_state={"scenario": scenario})

summary = await panel_graph.get_state("summary")
print(f"\n📜 Final Consensus:\n{summary}")



## Exercises

### Exercise 1: The Critic Loop
Create a workflow where:
1. **Creator Agent** generates a short story.
2. **Critic Agent** reviews it and provides feedback.
3. **Refiner Agent** rewrites the story based on feedback.

Use `GraphState` to pass the story and feedback.



In [ ]:
# Your code here
# 1. Define Agents (Creator, Critic, Refiner)
# 2. Define Nodes
# 3. Connect them: Creator >> Critic >> Refiner
# 4. Run it



### Exercise 2: Translation Voter
Create a parallel workflow where:
1. Three different Translation Agents translate a sentence (e.g., one formal, one casual, one poetic).
2. A Judge Agent picks the best one.

*Hint: This is similar to the Panel of Experts example.*



In [ ]:
# Your code here



## Summary

You've learned powerful patterns for Multi-Agent Workflows:

1. **Sequential**: Chaining agents to break down complex tasks (Researcher -> Writer).
2. **Routing**: Using a specialized agent to direct traffic (Triage -> Support).
3. **Parallel**: Running multiple agents at once for diverse perspectives (Experts -> Aggregator).
4. **Coordination**: Using `GraphState` as a shared memory for the team.

These patterns allow you to build robust, scalable AI systems that are far more capable than a single prompt.
